In [ ]:
""" ConvNet Implementation """

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torch.nn.functional as F

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyperparameters
num_epochs = 200
batch_size = 128
learning_rate = 0.01
momentum = 0.9
weight_decay = 1e-4

# Data augmentation and normalization for training
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# Only normalization for testing
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# Custom CNN model
class CustomCNN(nn.Module):
    def __init__(self):
        super(CustomCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.conv5 = nn.Conv2d(512, 512, kernel_size=3, padding=1)

        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.5)

        self.fc1 = nn.Linear(512 * 1 * 1, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 10)

        self.batch_norm1 = nn.BatchNorm2d(64)
        self.batch_norm2 = nn.BatchNorm2d(128)
        self.batch_norm3 = nn.BatchNorm2d(256)
        self.batch_norm4 = nn.BatchNorm2d(512)
        self.batch_norm5 = nn.BatchNorm2d(512)

    def forward(self, x):
        x = self.pool(F.relu(self.batch_norm1(self.conv1(x))))
        x = self.pool(F.relu(self.batch_norm2(self.conv2(x))))
        x = self.pool(F.relu(self.batch_norm3(self.conv3(x))))
        x = self.pool(F.relu(self.batch_norm4(self.conv4(x))))
        x = self.pool(F.relu(self.batch_norm5(self.conv5(x))))

        x = x.view(-1, 512 * 1 * 1)

        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.fc3(x)
        return x

model = CustomCNN().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum, weight_decay=weight_decay)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)

# Training loop
def train(model, train_loader, criterion, optimizer, scheduler, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if (i+1) % 100 == 0:
                print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

        scheduler.step(running_loss)

train(model, train_loader, criterion, optimizer, scheduler, num_epochs)

# Testing loop
def test(model, test_loader):
    model.eval()
    with torch.no_grad():
        n_correct = 0
        n_samples = 0
        class_correct = [0 for _ in range(10)]
        class_samples = [0 for _ in range(10)]

        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            n_samples += labels.size(0)
            n_correct += (predicted == labels).sum().item()

            for i in range(len(labels)):
                label = labels[i]
                pred = predicted[i]
                if label == pred:
                    class_correct[label] += 1
                class_samples[label] += 1

        acc = 100.0 * n_correct / n_samples
        print(f'Accuracy of the network: {acc:.2f}%')

        for i in range(10):
            if class_samples[i] != 0:
                acc = 100.0 * class_correct[i] / class_samples[i]
                print(f'Accuracy of {classes[i]}: {acc:.2f}%')
            else:
                print(f'Accuracy of {classes[i]}: N/A (no samples)')

test(model, test_loader)

100%|██████████| 170498071/170498071 [00:03<00:00, 51634560.17it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Epoch [1/200], Step [100/391], Loss: 1.7967
Epoch [1/200], Step [200/391], Loss: 1.7288
Epoch [1/200], Step [300/391], Loss: 1.3527
Epoch [2/200], Step [100/391], Loss: 1.3373
Epoch [2/200], Step [200/391], Loss: 1.1826
Epoch [2/200], Step [300/391], Loss: 1.2976
Epoch [3/200], Step [100/391], Loss: 1.0463
Epoch [3/200], Step [200/391], Loss: 1.1074
Epoch [3/200], Step [300/391], Loss: 1.3151
Epoch [4/200], Step [100/391], Loss: 1.1604
Epoch [4/200], Step [200/391], Loss: 1.0295
Epoch [4/200], Step [300/391], Loss: 1.0698
Epoch [5/200], Step [100/391], Loss: 0.9765
Epoch [5/200], Step [200/391], Loss: 0.9706
Epoch [5/200], Step [300/391], Loss: 1.0711
Epoch [6/200], Step [100/391], Loss: 0.9845
Epoch [6/200], Step [200/391], Loss: 0.9290
Epoch [6/200], Step [300/391], Loss: 0.7957
Epoch [7/200], Step [100/391], Loss: 0.9795
Epoch [7/200], Step [200/391], Loss: 0.7156
Epoch [7/200], Step [300/391], Loss: 0.8171
Epoch [8/200], Step [100/391], Loss: 0.8092
Epoch [8/200], Step [200/391], L